# Data Exploration

In [6]:
%run setup.py

time: 48.9 s (started: 2021-08-07 13:38:31 +00:00)


In [7]:
train.head()

,pickup_date,total_rides,total_takings
0,2013-08-01,307272,4.567414e+06
1,2013-08-02,236761,3.534616e+06
2,2013-08-03,226554,3.182267e+06
3,2013-08-04,202310,3.073609e+06
4,2013-08-05,406487,6.657713e+06


time: 8.19 ms (started: 2021-08-07 13:39:20 +00:00)


## MLFlow Setup

In [8]:
import mlflow

time: 1.57 s (started: 2021-08-07 13:39:20 +00:00)


In [9]:
mlflow.set_tracking_uri("http://192.168.64.21:5000/")
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

Current tracking uri: http://192.168.64.21:5000/
time: 460 µs (started: 2021-08-07 13:39:21 +00:00)


The taxi_daily_forecast experiment has already been created

In [12]:
train.head(5)

,pickup_date,total_rides,total_takings
0,2013-08-01,307272,4.567414e+06
1,2013-08-02,236761,3.534616e+06
2,2013-08-03,226554,3.182267e+06
3,2013-08-04,202310,3.073609e+06
4,2013-08-05,406487,6.657713e+06


time: 6.13 ms (started: 2021-08-07 13:41:50 +00:00)


In [13]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

time: 486 ms (started: 2021-08-07 13:45:04 +00:00)


In [ ]:
# forescasting horizon
# window_length
# step_length

In [14]:
SlidingWindowSplitter(fh=7, window_length=28, step_length=7)

SlidingWindowSplitter(fh=7, step_length=7, window_length=28)

time: 2.47 ms (started: 2021-08-07 13:47:37 +00:00)
